# Alex Spence
# DSC 630 Week 3
# Assignment 3.2: Using Data to Improve MLB Attendance

In this assignment, you will be using data on the Los Angeles Dodgers Major League Baseball (MLB) team located here: dodgers.csv. Use this data to make a recommendation to management on how to improve attendance. Tell a story with your analysis and clearly explain the steps you take to arrive at your conclusion. This is an open-ended question, and there is no one right answer. You are welcome to do additional research and/or use domain knowledge to assist your analysis, but clearly state any assumptions you make.

# Step 1: Loading and Exploring the Data

In [9]:
import statsmodels.api as sm
import pandas as pd

df = pd.read_csv('dodgers-2022.csv')

# Clean minor issues: convert numeric columns and strip spaces
df['attend'] = pd.to_numeric(df['attend'])
df['temp'] = pd.to_numeric(df['temp'])
df['skies'] = df['skies'].str.strip()

# Compute summary statistics
attendance_summary = df['attend'].describe()

# Print results
print("Attendance Summary Statistics:")
print(attendance_summary)

Attendance Summary Statistics:
count       81.000000
mean     41040.074074
std       8297.539460
min      24312.000000
25%      34493.000000
50%      40284.000000
75%      46588.000000
max      56000.000000
Name: attend, dtype: float64


Load the data from the CSV file into a Pandas DataFrame and perform basic cleaning. Ensuring numeric types for attendance and temperature, stripping extra spaces from categorical fields like 'skies'. 
<br>
<br>
The summary statistics provide an overview: 81 games, mean attendance of 41,040, standard deviation of 8,298, minimum of 24,312, and maximum of 56,000 (sellout). 
<br>
<br>
This indicates strong overall attendance (about 73% of stadium capacity on average) but opportunities to boost lower-attendance games. 
<br>
<br>
The median (40,284) is close to the mean, suggesting a symmetric distribution without extreme skew.

# Step 2: Grouping and Comparing Key Factors

In [3]:
# Group by day of the week
day_of_week_avg = df.groupby('day_of_week')['attend'].mean().sort_values(ascending=False)

# Group by month
month_avg = df.groupby('month')['attend'].mean().sort_values(ascending=False)

# Group by opponent
opponent_avg = df.groupby('opponent')['attend'].mean().sort_values(ascending=False)

# Day vs. Night
day_night_avg = df.groupby('day_night')['attend'].mean()

# Skies
skies_avg = df.groupby('skies')['attend'].mean()

# Temperature correlation
temp_corr = df['attend'].corr(df['temp'])

# Print results
print("\nAverage Attendance by Day of Week:")
print(day_of_week_avg)
print("\nAverage Attendance by Month:")
print(month_avg)
print("\nAverage Attendance by Opponent:")
print(opponent_avg)
print("\nAverage Attendance by Day/Night:")
print(day_night_avg)
print("\nAverage Attendance by Skies:")
print(skies_avg)
print(f"\nCorrelation between Temperature and Attendance: {temp_corr:.3f}")


Average Attendance by Day of Week:
day_of_week
Tuesday      47741.230769
Saturday     43072.923077
Sunday       42268.846154
Thursday     40407.400000
Friday       40116.923077
Wednesday    37585.166667
Monday       34965.666667
Name: attend, dtype: float64

Average Attendance by Month:
month
JUN    47940.444444
JUL    43884.250000
AUG    42751.533333
APR    39591.916667
SEP    38955.083333
MAY    37345.722222
OCT    36703.666667
Name: attend, dtype: float64

Average Attendance by Opponent:
opponent
Angels       49777.333333
Mets         49586.250000
Nationals    49267.333333
White Sox    46382.000000
Cubs         44206.666667
Padres       42092.222222
Phillies     41897.000000
Cardinals    40853.285714
Marlins      40665.333333
Reds         40649.000000
Rockies      39631.222222
Snakes       39315.444444
Giants       39296.333333
Pirates      38019.000000
Astros       35383.333333
Brewers      35358.750000
Braves       32245.000000
Name: attend, dtype: float64

Average Attendance by 

Grouped attendance by key categorical factors to identify patterns. 
<br>
<br>
Tuesdays (47,741) and weekends outperform weekdays, possibly due to promotions or scheduling. 
<br>
<br>
Summer months (June: 47,940) draw more fans than spring (April: 39,592) or fall, aligning with vacation periods and warmer weather. 
<br>
<br>
Rival games (e.g., Angels: 49,777) boost crowds compared to others (e.g., Braves: 32,245). 
<br>
<br>
Day games (41,793) slightly edge night games, and clear skies (41,729) beat cloudy (38,791). 
<br>
<br>
Temperature shows a weak positive correlation (r=0.099), meaning it's not a major driver in LA's mild climate. 
<br>
<br>
These groupings highlight factors influencing turnout.

# Step 3: Analyzing Promotions

In [4]:
# List of promotion columns
promos = ['cap', 'shirt', 'fireworks', 'bobblehead']

# Dictionary to store averages and counts
promo_analysis = {}

for promo in promos:
    # Average attendance with/without promo
    avg_yes = df[df[promo] == 'YES']['attend'].mean()
    avg_no = df[df[promo] == 'NO']['attend'].mean()
    count_yes = df[df[promo] == 'YES'].shape[0]
    promo_analysis[promo] = {
        'YES': {'avg': avg_yes, 'count': count_yes},
        'NO': {'avg': avg_no}
    }

# Print results
print("\nPromotion Analysis:")
for promo, data in promo_analysis.items():
    print(f"{promo.capitalize()}: YES (Avg: {data['YES']['avg']:.0f}, Count: {data['YES']['count']}) | NO (Avg: {data['NO']['avg']:.0f})")


Promotion Analysis:
Cap: YES (Avg: 38190, Count: 2) | NO (Avg: 41112)
Shirt: YES (Avg: 46644, Count: 3) | NO (Avg: 40825)
Fireworks: YES (Avg: 41078, Count: 14) | NO (Avg: 41032)
Bobblehead: YES (Avg: 53145, Count: 11) | NO (Avg: 39138)


I decided to focus on promotions here since management can directly influence this. 
<br>
<br>
Bobbleheads show the strongest boost (+14,007 fans on average, used in 11 games), pushing attendance near sellouts. 
<br>
<br>
Shirts (+5,819, 3 games) provide a modest lift, while fireworks (negligible difference, 14 games) and caps (-2,922, 2 games) show mixed or no clear 
impact. 
<br>
<br>
The fireworks effect may be diluted as they're often on Fridays which already have high attendance. The small sample sizes for some promotions limit conclusions, but bobbleheads seem to really be driving attendance increase.

# Step 4: Regression Analysis for Deeper Insights

In [12]:
import statsmodels.api as sm

# Prepare data for regression: convert promotions to binary (1/0)
df_reg = df.copy()
for promo in promos:
    df_reg[promo] = (df_reg[promo] == 'YES').astype(int)

# Create dummy variables for categoricals
df_reg = pd.get_dummies(df_reg, columns=['month', 'day_of_week', 'opponent', 'skies', 'day_night'], drop_first=True)

# Drop non-predictor columns (e.g., day is redundant)
df_reg = df_reg.drop(['day'], axis=1)

# Define X (predictors) and y (target)
X = df_reg.drop('attend', axis=1)
X = sm.add_constant(X)  # Add intercept
X = X.astype(float)  # Ensure all are numeric
y = df_reg['attend']

# Fit OLS model
model = sm.OLS(y, X).fit()

# Print summary
print("\nRegression Summary:")
print(model.summary())


Regression Summary:
                            OLS Regression Results                            
Dep. Variable:                 attend   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.482
Method:                 Least Squares   F-statistic:                     3.127
Date:                Tue, 23 Sep 2025   Prob (F-statistic):           0.000187
Time:                        21:01:21   Log-Likelihood:                -795.41
No. Observations:                  81   AIC:                             1663.
Df Residuals:                      45   BIC:                             1749.
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const    

To isolate effects while controlling for confounders, we use OLS regression with attendance as the dependent variable. 
<br>
<br>
Predictors include temperature, binary promotions, and dummies for month, day, opponent, etc. 
<br>
<br>
The model (R²=0.709) explains 71% of variance, though adjusted R² (0.482) suggests some overfit due to many predictors (35) relative to observations (81). 
<br>
<br>
Significant factors: bobbleheads (+9,718 fans, p=0.004), fireworks (+20,632, p=0.017—stronger than raw averages suggest), and certain days (e.g., Tuesday +26,490 vs. Friday baseline).
<br>
<br>
Opponents and months are mostly insignificant after controls, but rivals remain positive. This confirms promotions' causal potential, beyond correlations from earlier steps.

# Recommendations to Management
1) The easiest win should be to expand Bobblehead Giveaways. The data shows a roughly +9,718 fan boost. Increase the number of giveaway days from 11 to 20 per season. Plan them on the currently lowest attendance days to boost those (Monday and Wednesday). Estimated revenue increase: +$485,900/game at $50/ticket average.
<br>
2) Try to do more fireworks on those lower attended days of the week.
<br>
3) Add  more promotions in April/May and September/October the slowest times and promote midweek games aggressively.
<br>
4) Try new promotions and run testing on those.
<br>
All of the together could increase average attendance to 45,000+, adding millions in revenue.